## Experiments for binary treatment effect estimation comparison

In [1]:
import sys, os

# add the project root to sys.path
root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root not in sys.path:
    sys.path.insert(0, root)

from data_causl.utils import *
from data_causl.data import *
from frengression import *

device = torch.device('cpu')

import CausalEGM as cegm
# import the module
from models import *

import numpy as np
import pickle
import os
from tqdm import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)


np.random.seed(42)
n_tr = 1000
n_p = 1000


## Twins

In [2]:
params = {"dataset": "Semi_Twins",
    "output_dir": './',
    "v_dim": 50,
    "z_dims": [1,1,1,7],
    "x_min": 0,
    "x_max": 3,
    "lr": 0.0002,
    "bs": 32,
    "alpha": 1,
    "beta": 1,
    "gamma": 10,
    "g_d_freq": 5,
    "g_units": [64,64,64,64,64],
    "e_units": [64,64,64,64,64],
    "f_units": [64,32,8],
    "h_units": [64,32,8],
    "dz_units": [64,32,8],
    "dv_units": [64,32,8],
    "binary_treatment": False,
    "use_z_rec": True,
    "use_v_gan": False,
    "save_res": False,
    "save_model": False,}
# Set up the range of x values for predictions
x_vals = np.arange(0, 6, 0.1)
x_tensor = torch.tensor(x_vals.reshape(-1, 1), dtype=torch.float32)

In [ ]:
# Number of repetitions
nrep = 30
n_tr = 1000
num_iters = 1000
binary_intervention = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize storage for predictions
fr_preds = [[], [], []]  # 2.5%, mean, 97.5%
cegm_preds = []

# Ground truth ADRF values
ground_truth_adrf = Semi_Twins_adrf(x_vals)

# Begin experiment
for rep in tqdm(range(nrep)):
    # Simulate training data

    x_train, y_train, z_train = Semi_Twins_sampler(path = "../data_causl"
    ).load_all()
    
    # Convert to tensors
    x_tr = torch.tensor(x_train, dtype=torch.float32)
    y_tr = torch.tensor(y_train, dtype=torch.float32)
    z_tr = torch.tensor(z_train, dtype=torch.float32)
    
    # --- Frengression Model ---
    # Initialize and train
    fr_model = Frengression(
        x_tr.shape[1], y_tr.shape[1], z_tr.shape[1],
        noise_dim=1, num_layer=3, hidden_dim=100,
        device=device, x_binary=binary_intervention
    )
    fr_model.train_y(x_tr, z_tr, y_tr, num_iters=num_iters, lr=1e-4, print_every_iter=400)

    # Predict using Frengression model (quantiles and mean)
    fr_pred = fr_model.predict_causal(
        x_tensor, target=[0.025, "mean", 0.975], sample_size=1000
    )  # Returns a list of 3 tensors, each (30, 1)
    
    # Append predictions to the respective lists
    for i, quantile_pred in enumerate(fr_pred):
        fr_preds[i].append(quantile_pred.numpy().flatten())  # Convert to NumPy array and flatten


    # --- CausalEGM Model ---
    cegm_model = CausalEGM(params=params, random_seed=123 + rep)
    cegm_model.train(data=[x_train, y_train, z_train], n_iter=1000, verbose=False)

    # Predict using CausalEGM model
    cegm_pred = cegm_model.getADRF(x_list=x_vals.tolist())
    cegm_preds.append(cegm_pred)

# Convert predictions to NumPy arrays
fr_preds = [np.array(pred_list).T for pred_list in fr_preds]  # Each becomes (30, nrep)
cegm_preds = np.array(cegm_preds).T  # Convert to (30, nrep)

# Compute mean predictions and CIs for Frengression
fr_ci_lower = fr_preds[0].mean(axis=1)  # Mean of 2.5% predictions
fr_mean_pred = fr_preds[1].mean(axis=1)  # Mean predictions
fr_ci_upper = fr_preds[2].mean(axis=1)  # Mean of 97.5% predictions

# Compute mean predictions and CIs for CausalEGM
cegm_mean_pred = cegm_preds.mean(axis=1)
cegm_ci_lower = np.quantile(cegm_preds, 0.025, axis=1)
cegm_ci_upper = np.quantile(cegm_preds, 0.975, axis=1)


In [8]:
import os
import json
import numpy as np

# --- after you’ve computed these arrays --- #
# x_vals: array of dose levels, shape (n_doses,)
# fr_ci_lower, fr_mean_pred, fr_ci_upper: each shape (n_doses,)
# cegm_ci_lower, cegm_mean_pred, cegm_ci_upper: each shape (n_doses,)

# 1) build a results dict of pure Python types
summary = {
    "x_vals": x_vals.tolist(),
    "Frengression": {
        "ci_lower": fr_ci_lower.tolist(),
        "mean":     fr_mean_pred.tolist(),
        "ci_upper": fr_ci_upper.tolist()
    },
    "CausalEGM": {
        "ci_lower": cegm_ci_lower.tolist(),
        "mean":     cegm_mean_pred.tolist(),
        "ci_upper": cegm_ci_upper.tolist()
    }
}

# 2) make sure the folder exists
output_dir = "result/continuous"
os.makedirs(output_dir, exist_ok=True)

# 3) write to JSON
outfile = os.path.join(output_dir, "twins.json")
with open(outfile, "w") as f:
    json.dump(summary, f, indent=4)

print(f"Saved ADRF summary to {outfile}")


Saved ADRF summary to result/continuous/twins.json


## Hiran & Imbens

In [3]:

params = {"dataset": "Sim_Hirano_Imbens",
"output_dir": './',
'v_dim': 200,
"z_dims": [1,1,1,7],
"x_min": 0,
"x_max": 6,
"lr": 0.0002,
"bs": 32,
"alpha": 1,
"beta": 1,
"gamma": 10,
"g_d_freq": 5,
"g_units": [64,64,64,64,64],
"e_units": [64,64,64,64,64],
"f_units": [64,32,8],
"h_units": [64,32,8],
"dz_units": [64,32,8],
"dv_units": [64,32,8],
"binary_treatment": False,
"use_z_rec": True,
"use_v_gan": False,
"save_res": False,
"save_model": False,}

# Set up the range of x values for predictions
x_vals = np.arange(0, 6, 0.1)
x_tensor = torch.tensor(x_vals.reshape(-1, 1), dtype=torch.float32)

In [4]:
# Number of repetitions
nrep = 30
n_tr = 1000
num_iters = 1000
binary_intervention = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize storage for predictions
fr_preds = [[], [], []]  # 2.5%, mean, 97.5%
cegm_preds = []

# Ground truth ADRF values
ground_truth_adrf = Semi_Twins_adrf(x_vals)

# Begin experiment
for rep in tqdm(range(nrep)):
    # Simulate training data

    x_train, y_train, z_train = Sim_Hirano_Imbens_sampler(
        N=n_tr, v_dim=200, seed=1024 + rep, batch_size=n_tr
    ).load_all()
    
    
    # Convert to tensors
    x_tr = torch.tensor(x_train, dtype=torch.float32)
    y_tr = torch.tensor(y_train, dtype=torch.float32)
    z_tr = torch.tensor(z_train, dtype=torch.float32)
    
    # Truncate the dataset to x in (0, 3)
    mask = (x_train > 0) & (x_train < 3)  # `mask` has shape (1000,)
    mask = mask.flatten()  # Flatten to 1D if necessary
    x_tr = x_tr[mask]  # Apply mask to filter rows
    y_tr = y_tr[mask]
    z_tr = z_tr[mask]

    # --- Frengression Model ---
    # Initialize and train
    fr_model = Frengression(
        x_tr.shape[1], y_tr.shape[1], z_tr.shape[1],
        noise_dim=1, num_layer=3, hidden_dim=100,
        device=device, x_binary=binary_intervention
    )
    fr_model.train_y(x_tr, z_tr, y_tr, num_iters=num_iters, lr=1e-4, print_every_iter=400)

    # Predict using Frengression model (quantiles and mean)
    fr_pred = fr_model.predict_causal(
        x_tensor, target=[0.025, "mean", 0.975], sample_size=1000
    )  # Returns a list of 3 tensors, each (30, 1)
    
    # Append predictions to the respective lists
    for i, quantile_pred in enumerate(fr_pred):
        fr_preds[i].append(quantile_pred.numpy().flatten())  # Convert to NumPy array and flatten


    # --- CausalEGM Model ---
    cegm_model = CausalEGM(params=params, random_seed=123 + rep)
    cegm_model.train(data=[x_train[mask], y_train[mask], z_train[mask]], n_iter=1000, verbose=False)

    # Predict using CausalEGM model
    cegm_pred = cegm_model.getADRF(x_list=x_vals.tolist())
    cegm_preds.append(cegm_pred)

# Convert predictions to NumPy arrays
fr_preds = [np.array(pred_list).T for pred_list in fr_preds]  # Each becomes (30, nrep)
cegm_preds = np.array(cegm_preds).T  # Convert to (30, nrep)

# Compute mean predictions and CIs for Frengression
fr_ci_lower = fr_preds[0].mean(axis=1)  # Mean of 2.5% predictions
fr_mean_pred = fr_preds[1].mean(axis=1)  # Mean predictions
fr_ci_upper = fr_preds[2].mean(axis=1)  # Mean of 97.5% predictions

# Compute mean predictions and CIs for CausalEGM
cegm_mean_pred = cegm_preds.mean(axis=1)
cegm_ci_lower = np.quantile(cegm_preds, 0.025, axis=1)
cegm_ci_upper = np.quantile(cegm_preds, 0.975, axis=1)


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1: loss 2.4076,	loss_y 1.6749, 1.7086, 0.0674,	loss_eta 0.7327, 0.7681, 0.0708
Epoch 400: loss 1.1945,	loss_y 0.6099, 0.8586, 0.4975,	loss_eta 0.5846, 1.2082, 1.2471
Epoch 800: loss 0.7983,	loss_y 0.2010, 0.4053, 0.4085,	loss_eta 0.5972, 1.2573, 1.3202


2025-04-26 16:55:53.337144: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 16:55:53.337439: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1: loss 2.2632,	loss_y 1.5247, 1.5896, 0.1296,	loss_eta 0.7385, 0.7774, 0.0778
Epoch 400: loss 1.0539,	loss_y 0.5143, 1.0799, 1.1310,	loss_eta 0.5396, 1.1198, 1.1604
Epoch 800: loss 0.6284,	loss_y 0.0976, 0.2098, 0.2244,	loss_eta 0.5308, 1.1980, 1.3344


2025-04-26 16:56:41.614131: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 16:56:41.614475: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

...

2025-04-26 17:17:06.200777: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:17:06.201066: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1: loss 2.3750,	loss_y 1.6352, 1.6482, 0.0260,	loss_eta 0.7398, 0.7773, 0.0750
Epoch 400: loss 1.0821,	loss_y 0.5220, 0.8887, 0.7333,	loss_eta 0.5601, 1.1936, 1.2670
Epoch 800: loss 0.7368,	loss_y 0.1767, 0.3358, 0.3182,	loss_eta 0.5601, 1.2210, 1.3219


2025-04-26 17:17:55.517740: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:17:55.518073: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1: loss 2.2762,	loss_y 1.5365, 1.5741, 0.0753,	loss_eta 0.7397, 0.7794, 0.0793
Epoch 400: loss 1.1650,	loss_y 0.5729, 0.9084, 0.6710,	loss_eta 0.5921, 1.1699, 1.1557
Epoch 800: loss 0.7356,	loss_y 0.1737, 0.3684, 0.3894,	loss_eta 0.5619, 1.2424, 1.3609


2025-04-26 17:18:43.130180: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:18:43.130497: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

In [5]:
import os
import json
import numpy as np

# --- after you’ve computed these arrays --- #
# x_vals: array of dose levels, shape (n_doses,)
# fr_ci_lower, fr_mean_pred, fr_ci_upper: each shape (n_doses,)
# cegm_ci_lower, cegm_mean_pred, cegm_ci_upper: each shape (n_doses,)

# 1) build a results dict of pure Python types
summary = {
    "x_vals": x_vals.tolist(),
    "Frengression": {
        "ci_lower": fr_ci_lower.tolist(),
        "mean":     fr_mean_pred.tolist(),
        "ci_upper": fr_ci_upper.tolist()
    },
    "CausalEGM": {
        "ci_lower": cegm_ci_lower.tolist(),
        "mean":     cegm_mean_pred.tolist(),
        "ci_upper": cegm_ci_upper.tolist()
    }
}

# 2) make sure the folder exists
output_dir = "result/continuous"
os.makedirs(output_dir, exist_ok=True)

# 3) write to JSON
outfile = os.path.join(output_dir, "hiran.json")
with open(outfile, "w") as f:
    json.dump(summary, f, indent=4)

print(f"Saved ADRF summary to {outfile}")


Saved ADRF summary to result/continuous/hiran.json


## Colangelo

In [6]:
params = {"dataset": "Sim_Colangelo",
"output_dir": './',
'v_dim': 100,
"z_dims": [5,5,5,5],
"x_min": -2,
"x_max": 2,
"lr": 0.0002,
"bs": 32,
"alpha": 1,
"beta": 1,
"gamma": 10,
"g_d_freq": 5,
"g_units": [64,64,64,64,64],
"e_units": [64,64,64,64,64],
"f_units": [64,32,8],
"h_units": [64,32,8],
"dz_units": [64,32,8],
"dv_units": [64,32,8],
"binary_treatment": False,
"use_z_rec": True,
"use_v_gan": False,
"save_res": False,
"save_model": False,}

# Set up the range of x values for predictions
x_vals = np.arange(-4, 4, 0.1)
x_tensor = torch.tensor(x_vals.reshape(-1, 1), dtype=torch.float32)

In [12]:
from scipy.sparse import diags
from scipy.stats import norm
def Sim_Colangelo_sampler(N=20000, v_dim=100, seed=0,
                rho=0.5, offset = [-1,0,1], d=1, a=3, b=0.75):
    """Colangelo simulation dataset (continuous treatment) sampler (inherited from Base_sampler).

    Parameters
    ----------
    batch_size
        Int object denoting the batch size for mini-batch training. Default: ``32``.
    N
        Sample size. Default: ``20000``.
    v_dim
        Int object denoting the dimension for covariates. Default: ``200``.
    seed
        Int object denoting the random seed. Default: ``0``.
    Examples
    --------
    >>> from CausalEGM import Sim_Colangelo_sampler
    >>> ds = Sim_Colangelo_sampler(batch_size=32, N=20000, v_dim=100, seed=0)
    """

    np.random.seed(seed)
    k_list = [
        rho * np.ones(v_dim - 1),   # subdiagonal
        np.ones(v_dim),             # main diagonal
        rho * np.ones(v_dim - 1)    # superdiagonal
    ]
    offsets = [-1, 0, 1]
    sigma = diags(k_list, offsets).toarray()

    theta = np.array([(1/(l**2)) for l in list(range(1,(v_dim+1)))])
    epsilon = np.random.normal(0,1,N)
    nu = np.random.normal(0,1,N)
    v = np.random.multivariate_normal(np.zeros(v_dim),sigma,size=[N,])
    x = d*norm.cdf((a*v@theta)) + b*nu - 0.5
    y = 1.2*x + (x**3) + (x*v[:,0]) + 1.2*(v@theta) + epsilon
    x = x.reshape(-1,1)
    y = y.reshape(-1,1)
    return x, y, v
    

In [14]:
# Number of repetitions
nrep = 30
n_tr = 1000
num_iters = 1000
binary_intervention = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize storage for predictions
fr_preds = [[], [], []]  # 2.5%, mean, 97.5%
cegm_preds = []

# Ground truth ADRF values
ground_truth_adrf = Sim_Colangelo_adrf(x_vals)

# Begin experiment
for rep in tqdm(range(nrep)):
    # Simulate training data

    x_train, y_train, z_train = Sim_Colangelo_sampler(
       N=n_tr, v_dim=100, seed=42+rep
    )
    
    
    # Convert to tensors
    x_tr = torch.tensor(x_train, dtype=torch.float32)
    y_tr = torch.tensor(y_train, dtype=torch.float32)
    z_tr = torch.tensor(z_train, dtype=torch.float32)
    
    # Truncate the dataset to x in (0, 3)
    mask = (x_train > -2) & (x_train < 2)  # `mask` has shape (1000,)
    mask = mask.flatten()  # Flatten to 1D if necessary
    x_tr = x_tr[mask]  # Apply mask to filter rows
    y_tr = y_tr[mask]
    z_tr = z_tr[mask]

    # --- Frengression Model ---
    # Initialize and train
    fr_model = Frengression(
        x_tr.shape[1], y_tr.shape[1], z_tr.shape[1],
        noise_dim=1, num_layer=3, hidden_dim=100,
        device=device, x_binary=binary_intervention
    )
    fr_model.train_y(x_tr, z_tr, y_tr, num_iters=num_iters, lr=1e-4, print_every_iter=400)

    # Predict using Frengression model (quantiles and mean)
    fr_pred = fr_model.predict_causal(
        x_tensor, target=[0.025, "mean", 0.975], sample_size=1000
    )  # Returns a list of 3 tensors, each (30, 1)
    
    # Append predictions to the respective lists
    for i, quantile_pred in enumerate(fr_pred):
        fr_preds[i].append(quantile_pred.numpy().flatten())  # Convert to NumPy array and flatten


    # --- CausalEGM Model ---
    cegm_model = CausalEGM(params=params, random_seed=123 + rep)
    cegm_model.train(data=[x_train[mask], y_train[mask], z_train[mask]], n_iter=1000, verbose=False)

    cegm_pred = cegm_model.getADRF(x_list=x_vals.tolist())
    cegm_preds.append(cegm_pred)

# Convert predictions to NumPy arrays
fr_preds = [np.array(pred_list).T for pred_list in fr_preds]  # Each becomes (30, nrep)
cegm_preds = np.array(cegm_preds).T  # Convert to (30, nrep)

# Compute mean predictions and CIs for Frengression
fr_ci_lower = fr_preds[0].mean(axis=1)  # Mean of 2.5% predictions
fr_mean_pred = fr_preds[1].mean(axis=1)  # Mean predictions
fr_ci_upper = fr_preds[2].mean(axis=1)  # Mean of 97.5% predictions

# Compute mean predictions and CIs for CausalEGM
cegm_mean_pred = cegm_preds.mean(axis=1)
cegm_ci_lower = np.quantile(cegm_preds, 0.025, axis=1)
cegm_ci_upper = np.quantile(cegm_preds, 0.975, axis=1)


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1: loss 3.3969,	loss_y 2.6340, 2.6420, 0.0160,	loss_eta 0.7629, 0.8066, 0.0874
Epoch 400: loss 1.3480,	loss_y 0.7578, 1.5010, 1.4865,	loss_eta 0.5903, 1.2569, 1.3332
Epoch 800: loss 0.7544,	loss_y 0.2016, 0.3798, 0.3563,	loss_eta 0.5528, 1.1193, 1.1329


2025-04-26 17:36:34.822351: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:36:34.822913: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1: loss 3.3797,	loss_y 2.5991, 2.6154, 0.0326,	loss_eta 0.7806, 0.8247, 0.0881
Epoch 400: loss 1.4202,	loss_y 0.9124, 1.7344, 1.6441,	loss_eta 0.5078, 1.1552, 1.2949
Epoch 800: loss 0.8535,	loss_y 0.2942, 0.5528, 0.5171,	loss_eta 0.5593, 1.0717, 1.0248


2025-04-26 17:37:24.121047: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:37:24.121323: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

...

2025-04-26 17:58:06.458539: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:58:06.458835: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1: loss 3.2784,	loss_y 2.4897, 2.5339, 0.0882,	loss_eta 0.7886, 0.8295, 0.0817
Epoch 400: loss 1.5725,	loss_y 1.0156, 1.8542, 1.6772,	loss_eta 0.5569, 1.2025, 1.2911
Epoch 800: loss 0.8464,	loss_y 0.2774, 0.5156, 0.4765,	loss_eta 0.5691, 1.1109, 1.0837


2025-04-26 17:58:55.468703: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:58:55.469018: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1: loss 3.4579,	loss_y 2.6886, 2.7198, 0.0624,	loss_eta 0.7693, 0.8122, 0.0858
Epoch 400: loss 1.7971,	loss_y 1.1925, 1.9998, 1.6144,	loss_eta 0.6046, 1.2519, 1.2946
Epoch 800: loss 1.1463,	loss_y 0.5822, 1.1649, 1.1655,	loss_eta 0.5641, 1.1228, 1.1173


2025-04-26 17:59:39.510484: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 17:59:39.510786: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

In [15]:
summary = {
    "x_vals": x_vals.tolist(),
    "Frengression": {
        "ci_lower": fr_ci_lower.tolist(),
        "mean":     fr_mean_pred.tolist(),
        "ci_upper": fr_ci_upper.tolist()
    },
    "CausalEGM": {
        "ci_lower": cegm_ci_lower.tolist(),
        "mean":     cegm_mean_pred.tolist(),
        "ci_upper": cegm_ci_upper.tolist()
    }
}


output_dir = "result/continuous"
os.makedirs(output_dir, exist_ok=True)

outfile = os.path.join(output_dir, "colangelo.json")
with open(outfile, "w") as f:
    json.dump(summary, f, indent=4)

print(f"Saved ADRF summary to {outfile}")


Saved ADRF summary to result/continuous/colangelo.json


## Sun

In [ ]:
params = {"dataset": "Sim_Sun",
"output_dir": './',
'v_dim': 200,
"z_dims": [2,2,2,4],
"x_min": 0,
"x_max": 3,
"lr": 0.0002,
"bs": 32,
"alpha": 1,
"beta": 1,
"gamma": 10,
"g_d_freq": 5,
"g_units": [64,64,64,64,64],
"e_units": [64,64,64,64,64],
"f_units": [64,32,8],
"h_units": [64,32,8],
"dz_units": [64,32,8],
"dv_units": [64,32,8],
"binary_treatment": False,
"use_z_rec": True,
"use_v_gan": False,
"save_res": False,
"save_model": False,}

# Set up the range of x values for predictions
x_vals = np.arange(0, 6, 0.1)
x_tensor = torch.tensor(x_vals.reshape(-1, 1), dtype=torch.float32)

In [ ]:
# Number of repetitions
nrep = 30
n_tr = 1000
num_iters = 1000
binary_intervention = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize storage for predictions
fr_preds = [[], [], []]  # 2.5%, mean, 97.5%
cegm_preds = []

# Ground truth ADRF values
ground_truth_adrf = Sim_Sun_adrf(x_vals)

# Begin experiment
for rep in tqdm(range(nrep)):
    # Simulate training data

    x_train, y_train, z_train = cegm.Sim_Sun_sampler(
        N=n_tr, v_dim=200, seed=42 + rep, batch_size=n_tr
    ).load_all()
    
    
    # Convert to tensors
    x_tr = torch.tensor(x_train, dtype=torch.float32)
    y_tr = torch.tensor(y_train, dtype=torch.float32)
    z_tr = torch.tensor(z_train, dtype=torch.float32)
    
    # Truncate the dataset to x in (0, 3)
    mask = (x_train > 0) & (x_train < 3)  # `mask` has shape (1000,)
    mask = mask.flatten()  # Flatten to 1D if necessary
    x_tr = x_tr[mask]  # Apply mask to filter rows
    y_tr = y_tr[mask]
    z_tr = z_tr[mask]

    # --- Frengression Model ---
    # Initialize and train
    fr_model = Frengression(
        x_tr.shape[1], y_tr.shape[1], z_tr.shape[1],
        noise_dim=1, num_layer=3, hidden_dim=100,
        device=device, x_binary=binary_intervention
    )
    fr_model.train_y(x_tr, z_tr, y_tr, num_iters=num_iters, lr=1e-4, print_every_iter=400)

    # Predict using Frengression model (quantiles and mean)
    fr_pred = fr_model.predict_causal(
        x_tensor, target=[0.025, "mean", 0.975], sample_size=1000
    )  # Returns a list of 3 tensors, each (30, 1)
    
    # Append predictions to the respective lists
    for i, quantile_pred in enumerate(fr_pred):
        fr_preds[i].append(quantile_pred.numpy().flatten())  # Convert to NumPy array and flatten


    # --- CausalEGM Model ---
    cegm_model = CausalEGM(params=params, random_seed=123 + rep)
    cegm_model.train(data=[x_train[mask], y_train[mask], z_train[mask]], n_iter=1000, verbose=False)

    cegm_pred = cegm_model.getADRF(x_list=x_vals.tolist())
    cegm_preds.append(cegm_pred)

# Convert predictions to NumPy arrays
fr_preds = [np.array(pred_list).T for pred_list in fr_preds]  # Each becomes (30, nrep)
cegm_preds = np.array(cegm_preds).T  # Convert to (30, nrep)

# Compute mean predictions and CIs for Frengression
fr_ci_lower = fr_preds[0].mean(axis=1)  # Mean of 2.5% predictions
fr_mean_pred = fr_preds[1].mean(axis=1)  # Mean predictions
fr_ci_upper = fr_preds[2].mean(axis=1)  # Mean of 97.5% predictions

# Compute mean predictions and CIs for CausalEGM
cegm_mean_pred = cegm_preds.mean(axis=1)
cegm_ci_lower = np.quantile(cegm_preds, 0.025, axis=1)
cegm_ci_upper = np.quantile(cegm_preds, 0.975, axis=1)


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1: loss 3.3611,	loss_y 2.5769, 2.6293, 0.1049,	loss_eta 0.7843, 0.8190, 0.0694
Epoch 400: loss 1.4521,	loss_y 0.9167, 1.7593, 1.6852,	loss_eta 0.5354, 1.1901, 1.3095
Epoch 800: loss 0.7151,	loss_y 0.1864, 0.4250, 0.4773,	loss_eta 0.5288, 1.0586, 1.0598


  0%|          | 0/30 [00:03<?, ?it/s]


ValueError: in user code:

    File "/opt/anaconda3/lib/python3.11/site-packages/CausalEGM/causalEGM.py", line 206, in train_disc_step  *
        data_z_ = self.e_net(data_v)
    File "/opt/anaconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/lib/python3.11/site-packages/CausalEGM/model.py", line 40, in call
        x = fc_layer(inputs) if i==0 else fc_layer(x)

    ValueError: Exception encountered when calling BaseFullyConnectedNet.call().
    
    [1mInput 0 of layer "dense_1686" is incompatible with the layer: expected axis -1 of input shape to have value 200, but received input with shape (32, 100)[0m
    
    Arguments received by BaseFullyConnectedNet.call():
      • inputs=tf.Tensor(shape=(32, 100), dtype=float32)
      • training=True


In [ ]:
summary = {
    "x_vals": x_vals.tolist(),
    "Frengression": {
        "ci_lower": fr_ci_lower.tolist(),
        "mean":     fr_mean_pred.tolist(),
        "ci_upper": fr_ci_upper.tolist()
    },
    "CausalEGM": {
        "ci_lower": cegm_ci_lower.tolist(),
        "mean":     cegm_mean_pred.tolist(),
        "ci_upper": cegm_ci_upper.tolist()
    }
}


output_dir = "result/continuous"
os.makedirs(output_dir, exist_ok=True)

outfile = os.path.join(output_dir, "sun.json")
with open(outfile, "w") as f:
    json.dump(summary, f, indent=4)

print(f"Saved ADRF summary to {outfile}")
